In [1]:
import os
import pandas as pd
import shutil
import datetime as dt
import csv

In [2]:
today = dt.date.today().strftime("%d_%m_%Y")

In [3]:
directory = os.chdir('../')
df = pd.read_csv('sampledata.csv')
ADT_txt = './ADT_sample.txt'
ADT_csv = f'./Archive/Modified/ADT_{today}_Modified_file.csv'
ORU_csv = f'./Archive/Modified/ORU_{today}_Modified_file.csv'


# Examine our data

In [4]:
df.head()

,#,id,site_id,service_location,message_type,message_time,message_id,account_number,discharge_disposition,financial_class,...,bill_amount,patient_drivers_license_number,guarantor_first_name,guarantor_last_name,guarantor_middle_name,guarantor_address_1,guarantor_address_2,guarantor_city,guarantor_state,guarantor_zip
0,1,30056263,C22,1,ADT-A08,19:00.0,5885975,CA00001,NaN,PPO,...,700,NaN,Testing,MU,NaN,12345,SOMEWHERE,VALENCIA,CA,91355
1,2,30056267,C22,1,ADT-A08,44:00.0,5885976,CA00001,NaN,PPO,...,727,NaN,Testing,MU,NaN,12345,SOMEWHERE,VALENCIA,CA,91355
2,3,30056269,C22,1,ADT-A08,14:00.0,5885977,CA00001,NaN,PPO,...,728,NaN,Testing,MU,NaN,12345,SOMEWHERE,VALENCIA,CA,91355
3,4,30058986,C22,1,ADT-A08,30:00.0,5886054,CA00001,NaN,PPO,...,912,NaN,Testing,MU,NaN,12345,SOMEWHERE,VALENCIA,CA,91355
4,5,30630702,I99,1,ADT-A04,48:28.0,5886293,CA00003,NaN,Self-pay,...,984,NaN,QINEVAAcuteprone,Printegratedtest,A,951 S Hebron Ave,NaN,EVANSVILLE,IN,47714


# Check for files, make directories, and copy files to Original and Modified. Initialize lists of file/dir names.

In [5]:
orig_files = ['ADT_sample.txt', 'Sample_ORU.txt', 'sampledata.csv']
dirs_to_make = ['Archive', 'Archive/Original', 'Archive/Modified']
modified_path = 'Archive/Modified'


for dir in dirs_to_make:
    if not os.path.exists(dir):
        os.mkdir(dir)

files_exist = True
for file in orig_files:
    if not os.path.exists(file):
        files_exist = False
        print(f'File {file} does not exist. Ensure all files are present.')
        break
    else:
        print(f'File {file} exists. Copying file to Archive/Original/ folder...')
        shutil.copy(file, 'Archive/Original/')   

File ADT_sample.txt exists. Copying file to Archive/Original/ folder...
File Sample_ORU.txt exists. Copying file to Archive/Original/ folder...
File sampledata.csv exists. Copying file to Archive/Original/ folder...


# Function to create new CSVs based on specified prefix and add today's date to the filenames.

In [15]:
def create_csv(csv_name, destination):
        # Construct the filename
    csv_file_path = f'{csv_name}_{today}_Modified_file.csv'
    csv_full_path = os.path.join(destination, csv_file_path)
    return csv_full_path

create_csv('ADT',modified_path)
create_csv('ORU',modified_path)

ADT_csv = pd.read_csv(ADT_csv)
ORU_csv = pd.read_csv(ORU_csv)

'Archive/Modified/ORU_17_08_2023_Modified_file.csv'

# Function to parse given CSV and identify patients by message_type using first string before '_' and match that to the appropriate csv and create a column in each csv with message_type and the patient info.

In [17]:
def csv_parser(df, mod_csv):
    # Filter the input dataframe based on the message_type
    match_str = mod_csv.split("_")[0].split("/")[-1]
    message_category = df[df['message_type'].str.startswith(match_str)]
    
    # Check if mod_csv exists and has data
    if os.path.exists(mod_csv) and os.path.getsize(mod_csv) > 0:
        df_mod = pd.read_csv(mod_csv)
    else:
        # If mod_csv doesn't exist or is empty, create an empty dataframe with the same columns as df
        df_mod = pd.DataFrame(columns=df.columns)
    
    # Append the filtered data from df to df_mod
    df_mod = df_mod.append(message_category, ignore_index=True)
    
    # Save the concatenated data back to mod_csv
    df_mod.to_csv(mod_csv, index=False)


csv_parser(df, ADT_csv)
csv_parser(df, ORU_csv)




AttributeError: 'DataFrame' object has no attribute 'append'

In [ ]:
df_adt = pd.read_csv(ADT_csv)
df_oru = pd.read_csv(ORU_csv)


In [ ]:
df_oru

,#,id,site_id,service_location,message_type,message_time,message_id,account_number,discharge_disposition,financial_class,...,bill_amount,patient_drivers_license_number,guarantor_first_name,guarantor_last_name,guarantor_middle_name,guarantor_address_1,guarantor_address_2,guarantor_city,guarantor_state,guarantor_zip
0,121,2231379,I71,1,ORU-R01,57:35.0,3358130,CA00001,NaN,P,...,661,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TN,NaN


# Parse Messages (NEED TO ADD MORE FIELDS IF POSSIBLE TO THESE)

In [ ]:
def extract_msg_fields(message):
    with open(message, 'r') as file_3:
        message = file_3.read()


        # Split the message by line
        lines = message.split('\n')
        
        # Define a dictionary to hold the extracted values
        extracted_data = {}
        
        # Extract data from PID segment
        pid_segment = [line for line in lines if line.startswith('PID')]
        if pid_segment:
            pid_fields = pid_segment[0].split('|')
            extracted_data['patient_last_name'] = pid_fields[5].split('^')[0] if len(pid_fields) > 5 else None
            extracted_data['patient_first_name'] = pid_fields[5].split('^')[1] if len(pid_fields) > 5 else None
            extracted_data['patient_middle_name'] = pid_fields[5].split('^')[2] if len(pid_fields) > 5 and len(pid_fields[5].split('^')) > 2 else None
            extracted_data['patient_address'] = pid_fields[11].split('^')[0] if len(pid_fields) > 11 else None
            extracted_data['state'] = pid_fields[11].split('^')[3] if len(pid_fields) > 11 and len(pid_fields[11].split('^')) > 3 else None
            extracted_data['account_number'] = pid_fields[3] if len(pid_fields) > 3 else None
            extracted_data['bill_amount'] = 1234
            extracted_data['date_of_service'] = today
            print(extracted_data)
        return extracted_data


# Extract fields from the message content
ORU_message = extract_msg_fields("./Archive/Original/Sample_ORU.txt")








{'patient_last_name': 'DUCK', 'patient_first_name': 'DONALD', 'patient_middle_name': 'L', 'patient_address': '1111 MATTERHORN RD', 'state': 'FL', 'account_number': 'SA00401418', 'bill_amount': 1234, 'date_of_service': '17_08_2023'}


In [ ]:
# copy the ORU_message to the existing df_oru dataframe with the keys mapping to the columns in the dataframe
df_oru = pd.concat([df_oru, pd.DataFrame([ORU_message])], ignore_index=True)
df_oru

,#,id,site_id,service_location,message_type,message_time,message_id,account_number,discharge_disposition,financial_class,...,guarantor_last_name,guarantor_middle_name,guarantor_address_1,guarantor_address_2,guarantor_city,guarantor_state,guarantor_zip,patient_address,state,date_of_service
0,121.0,2231379.0,I71,1.0,ORU-R01,57:35.0,3358130.0,CA00001,NaN,P,...,NaN,NaN,NaN,NaN,NaN,TN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SA00401418,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1111 MATTERHORN RD,FL,17_08_2023


In [ ]:
ADT_message = extract_msg_fields("./Archive/Original/ADT_sample.txt")
df_adt = pd.concat([df_adt, pd.DataFrame([ADT_message])], ignore_index=True)
df_adt

{'patient_last_name': 'MOUSE', 'patient_first_name': 'MICKEY', 'patient_middle_name': 'LITTLE', 'patient_address': '100 WALDO AVE', 'state': 'CA', 'account_number': '1395357', 'bill_amount': 1234, 'date_of_service': '17_08_2023'}


,#,id,site_id,service_location,message_type,message_time,message_id,account_number,discharge_disposition,financial_class,...,guarantor_last_name,guarantor_middle_name,guarantor_address_1,guarantor_address_2,guarantor_city,guarantor_state,guarantor_zip,patient_address,state,date_of_service
0,1.0,30056263.0,C22,1.0,ADT-A08,19:00.0,5885975.0,CA00001,NaN,PPO,...,MU,NaN,12345,SOMEWHERE,VALENCIA,CA,91355,NaN,NaN,NaN
1,2.0,30056267.0,C22,1.0,ADT-A08,44:00.0,5885976.0,CA00001,NaN,PPO,...,MU,NaN,12345,SOMEWHERE,VALENCIA,CA,91355,NaN,NaN,NaN
2,3.0,30056269.0,C22,1.0,ADT-A08,14:00.0,5885977.0,CA00001,NaN,PPO,...,MU,NaN,12345,SOMEWHERE,VALENCIA,CA,91355,NaN,NaN,NaN
3,4.0,30058986.0,C22,1.0,ADT-A08,30:00.0,5886054.0,CA00001,NaN,PPO,...,MU,NaN,12345,SOMEWHERE,VALENCIA,CA,91355,NaN,NaN,NaN
4,5.0,30630702.0,I99,1.0,ADT-A04,48:28.0,5886293.0,CA00003,NaN,Self-pay,...,Printegratedtest,A,951 S Hebron Ave,NaN,EVANSVILLE,IN,47714,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
161,193.0,66088.0,I71,1.0,ADT-A02,28:53.0,3352205.0,CA00002,NaN,P,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
162,198.0,59157.0,I71,1.0,ADT-A04,05:10.0,3345345.0,CA00005,NaN,P,...,Upttest,NaN,NaN,NaN,NaN,IN,,NaN,NaN,NaN
163,199.0,59158.0,I71,1.0,ADT-A02,05:11.0,3345346.0,CA00006,NaN,P,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
164,200.0,59168.0,I71,1.0,ADT-A02,41:13.0,3345356.0,CA00007,NaN,P,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Create patient_name column using existing columns format: last, first, middle

In [ ]:
def name_concat(df):
    df['patient_name'] = df['patient_last_name'] + ', ' + df['patient_first_name'] + ' ' + df['patient_middle_name']
    return df

name_concat(df_adt)
name_concat(df_oru)

In [ ]:
print(df_adt['patient_name'].head(100))

df_adt.to_csv(ADT_csv, index=True)
df_oru.to_csv(ORU_csv, index=True)


0                                         NaN
1                                         NaN
2                                         NaN
3                                         NaN
4     Printegratedtest, QINEVAAcuteprone John
                       ...                   
95                    Uptest, Ufinmerge Clark
96                    Uptest, Ufinmerge Clark
97                    Uptest, Ufinmerge Clark
98                    Uptest, Ufinmerge Clark
99                                        NaN
Name: patient_name, Length: 100, dtype: object


In [ ]:
df_oru['patient_name']

0    Uptest, Finmerge Clark
1            DUCK, DONALD L
Name: patient_name, dtype: object

# Create report txt with total bill amount for each state and append row at end of bill_amount that sums up total bill

In [ ]:
def sum_bills(df):
    grouped_state_data = df[df['patient_state'].notna() & df['patient_state'].str.strip().ne('')]

    # Calculating the grouped sum
    grouped_sum = grouped_state_data.groupby('patient_state')['bill_amount'].sum()

    # Adding the total sum at the end
    total_sum = grouped_state_data['bill_amount'].sum()
    grouped_sum['Total'] = total_sum

    # Saving the result to a text file
    output_path = "./Archive/Modified/total_bill_sum.txt"
    grouped_sum.to_csv(output_path, header=['bill_amount'], sep='\t')
    


sum_bills(df)